# Generating reduce.dat with PCA and Decision Trees

So you've already got ready.dat, but you want to ditch her for the hotter, younger and slimmer reduce.dat?

Well, you've come to the right place!

In [1]:
from dataload import *

In [2]:
x_train, x_test, y_train, y_test = load_object('ready.dat')

In [16]:
print(x_train.shape,x_test.shape)
# yes, this is supposed to be.

(4408587, 56) (489844, 56)


First get list of columns that account for variance of minority attack types.

In [4]:
def cats_importance(cats_train, target):
    from sklearn.tree import DecisionTreeClassifier
    dt = DecisionTreeClassifier(max_depth=10, random_state=4129)
    dt.fit(cats_train, y_train.attack_type == target)
    return pd.DataFrame(dt.feature_importances_, columns=['imp']).sort_values(by='imp', axis=0, ascending=False).iloc[:20]
# [cats_importance(cats_train, i) for i in ['normal','dos','probe','r2l','u2r']]

In [5]:
r2l_impts = cats_importance(x_train[:,0:37], 'r2l')
u2r_impts = cats_importance(x_train[:,0:37], 'u2r')
t_percent_var = 0.0
col_list = []
for i in range(20):
    col_list.append(r2l_impts.index[i])
    t_percent_var += r2l_impts.iloc[i, 0]
    if(t_percent_var > 0.85):
        break
t_percent_var = 0.0
for i in range(20):
    if(u2r_impts.index[i] not in col_list):
        col_list.append(u2r_impts.index[i])
#    else:
#        print('duplicate')
    t_percent_var += u2r_impts.iloc[i, 0]
    if(t_percent_var > 0.85):
        break
col_list.sort()
print(col_list)

[0, 1, 2, 6, 10, 14, 17, 18, 27, 30, 31]


Next, top 5 PCA components

In [8]:
## get list of column that accounts for variance of minority attack types
## Get top 5 PCA components
from sklearn.decomposition import PCA
pca = PCA(n_components = 5, random_state = 4129)
pca_result = pca.fit_transform(x_train[:,0:37])
pca.explained_variance_ratio_

array([0.59967437, 0.2070158 , 0.12038367, 0.02078111, 0.01783307])

In [12]:
## Create new x_train_new with PCA component
x_train_new = pca_result.copy()
## Add list of minority attack types columns to x_train_new
for i in col_list:
    x_train_new = np.concatenate((x_train_new, x_train[:, i:i+1]), axis = 1)
## Add remaining columns not within first 37 of x_train
x_train_new = np.concatenate((x_train_new, x_train[:, 37:]), axis = 1)


In [13]:
x_train_new.shape

(4408587, 35)

In [17]:
## do same for x_test
pca_result_test = pca.transform(x_test[:,0:37])
x_test_new = pca_result_test.copy()
for i in col_list:
    x_test_new = np.concatenate((x_test_new, x_test[:, i:i+1]), axis = 1)
x_test_new = np.concatenate((x_test_new, x_test[:, 37:]), axis = 1)

x_test_new.shape
# the key question here is: are there features which are highly correlated with the rare classes?
# we should use such features directly in the classifiers -- don't pass them thru PCA.

(489844, 35)

In [18]:
save_object([x_train_new, x_test_new, y_train, y_test],'reduce.dat')